In [1]:
# Install required libraries
%pip install web3 py-solc-x

  Using cached web3-7.7.0-py3-none-any.whl.metadata (5.5 kB)
  Using cached py_solc_x-2.0.3-py3-none-any.whl.metadata (5.8 kB)
  Using cached eth_abi-5.2.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached eth_account-0.13.4-py3-none-any.whl.metadata (5.3 kB)
  Using cached eth_hash-0.7.1-py3-none-any.whl.metadata (4.2 kB)
  Using cached eth_typing-5.1.0-py3-none-any.whl.metadata (3.2 kB)
  Using cached eth_utils-5.2.0-py3-none-any.whl.metadata (5.4 kB)
  Using cached hexbytes-1.3.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached aiohttp-3.11.11-cp311-cp311-win_amd64.whl.metadata (8.0 kB)
  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached types_requests-2.32.0.20241016-py3-none-any.whl.metadata (1.9 kB)
  Using cached websockets-13.1-cp311-cp311-win_amd64.whl.metadata (7.0 kB)
  Using cached pyunormalize-16.0.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached packaging-23.2-py3-none-any.wh


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from web3 import Web3

# Connect to the local Ganache blockchain
ganache_url = "http://127.0.0.1:7545"  # Default Ganache RPC URL
web3 = Web3(Web3.HTTPProvider(ganache_url))

# Check the connection
if web3.is_connected():
    print("Connected to the Ganache blockchain")
else:
    print("Failed to connect")

Connected to the Ganache blockchain


In [3]:
import solcx

# List available versions
print(solcx.get_installable_solc_versions())

# Install a specific version
solcx.install_solc('0.8.6')

# Set the installed version as the default
solcx.set_solc_version('0.8.6')

[<Version('0.8.28')>, <Version('0.8.27')>, <Version('0.8.26')>, <Version('0.8.25')>, <Version('0.8.24')>, <Version('0.8.23')>, <Version('0.8.22')>, <Version('0.8.21')>, <Version('0.8.20')>, <Version('0.8.19')>, <Version('0.8.18')>, <Version('0.8.17')>, <Version('0.8.16')>, <Version('0.8.15')>, <Version('0.8.14')>, <Version('0.8.13')>, <Version('0.8.12')>, <Version('0.8.11')>, <Version('0.8.10')>, <Version('0.8.9')>, <Version('0.8.8')>, <Version('0.8.7')>, <Version('0.8.6')>, <Version('0.8.5')>, <Version('0.8.4')>, <Version('0.8.3')>, <Version('0.8.2')>, <Version('0.8.1')>, <Version('0.8.0')>, <Version('0.7.6')>, <Version('0.7.5')>, <Version('0.7.4')>, <Version('0.7.3')>, <Version('0.7.2')>, <Version('0.7.1')>, <Version('0.7.0')>, <Version('0.6.12')>, <Version('0.6.11')>, <Version('0.6.10')>, <Version('0.6.9')>, <Version('0.6.8')>, <Version('0.6.7')>, <Version('0.6.6')>, <Version('0.6.5')>, <Version('0.6.4')>, <Version('0.6.3')>, <Version('0.6.2')>, <Version('0.6.1')>, <Version('0.6.0')

In [4]:
import solcx
from solcx import compile_source

# Load and compile the Solidity contract
with open("Healthcare.sol", "r") as file:
    contract_source_code = file.read()

compiled_sol = compile_source(
    contract_source_code,
    output_values=["abi", "bin"]
)

# Extract ABI and bytecode
contract_id, contract_interface = compiled_sol.popitem()
abi = contract_interface["abi"]
bytecode = contract_interface["bin"]

# Deploy the contract
web3.eth.default_account = web3.eth.accounts[0]  # Use the first Ganache account
Healthcare = web3.eth.contract(abi=abi, bytecode=bytecode)

# Deploy transaction
tx_hash = Healthcare.constructor().transact()
tx_receipt = web3.eth.wait_for_transaction_receipt(tx_hash)

print(f"Contract deployed at address: {tx_receipt.contractAddress}")


Contract deployed at address: 0x09AC16c6f3fA0f21910021368D31A7B42BBB35a6


In [5]:
import json

# Save ABI to a JSON file
with open('Healthcare.abi.json', 'w') as f:
    json.dump(abi, f, indent=4)

print("ABI saved to Healthcare.abi.json")

ABI saved to Healthcare.abi.json


In [6]:
# Interact with the deployed contract
contract_address = tx_receipt.contractAddress
healthcare_contract = web3.eth.contract(address=contract_address, abi=abi)

# Add a new record
tx_hash = healthcare_contract.functions.addRecord("John Doe", "EncryptedData123").transact()
web3.eth.wait_for_transaction_receipt(tx_hash)

# Retrieve the record
record = healthcare_contract.functions.getRecord(1).call()
print("Record Details:", record)


Record Details: ['John Doe', 'EncryptedData123', '0x42F78a2db8619B47F1A4c280DEc19519AA969eb1']
